In [1]:
import os 
os.chdir("../")
os.getcwd()

'c:\\Users\\arunk\\FraudDetection'

In [2]:
from fraudDetection.logger import logging
from fraudDetection.exception import FraudDetectionException
from fraudDetection.config.configuration import ConfigurationManager
from fraudDetection.components import DataIngestion, DataValidation, DataTransformation,ModelTrainer
from fraudDetection.entity import DataIngestionArtifact, DataValidationArtifact, DataTransformationArtifact, ModelTrainerArtifact

from fraudDetection.constants import CONFIG_FILE_PATH

config = ConfigurationManager(CONFIG_FILE_PATH)
data_ingestion_config = config.get_data_ingestion_config()
data_ingestion_artifacts = DataIngestion(data_ingestion_config).initiate_data_ingestion()

data_validation_config =config.get_data_validation_config()
data_validation_artifacts =DataValidation(data_ingestion_artifacts,data_validation_config).initiate_data_validation()
data_transformation_config =config.get_data_transformation_config()
data_transformation_artifacts = DataTransformation(data_validation_artifacts,data_ingestion_artifacts,data_transformation_config).initiate_data_transformation()

c:\Users\arunk\FraudDetection\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


====>>>File already exists


C:\Users\arunk\FraudDetection\src\fraudDetection\components\data_transformation.py:253: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.fillna(df.median(),inplace=True)
C:\Users\arunk\FraudDetection\src\fraudDetection\components\data_transformation.py:253: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.fillna(df.median(),inplace=True)


In [8]:
from fraudDetection.utils import load_numpy_array_data
from pathlib import Path

train_path = Path(data_transformation_artifacts.transformed_train_file_path)
train_path


WindowsPath('c:/Users/arunk/FraudDetection/artifacts/transformed_dir/train/file_0.npz')

In [10]:
train_arr = load_numpy_array_data(train_path)

X_train, y_train = train_arr[:,:-1], train_arr[:,-1]

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [14]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
clf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
clf_random = RandomizedSearchCV(estimator = clf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
clf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [24]:
print(clf_random.best_params_)
best_model = clf_random.best_estimator_     

{'n_estimators': 600, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 60, 'bootstrap': False}


RandomForestClassifier(bootstrap=False, max_depth=60, max_features='sqrt',
                       min_samples_split=5, n_estimators=600)

In [25]:
test_arr = load_numpy_array_data(Path(data_transformation_artifacts.transformed_test_file_path))
X_test, y_test = test_arr[:,:-1], test_arr[:,-1]
y_pred = clf_random.predict(X_test)


In [37]:
clf_random.best_params_

{'n_estimators': 600,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 60,
 'bootstrap': False}

In [33]:
from sklearn.metrics import f1_score, precision_recall_curve, recall_score, classification_report, fbeta_score, precision_score ,accuracy_score, balanced_accuracy_score
metrics = [f1_score, precision_recall_curve, recall_score, classification_report, precision_score ,accuracy_score, balanced_accuracy_score]
scores = {}
for metric in metrics:
    score = metric(y_test,y_pred)
    scores[metric.__name__] = score
  
f1_score = scores['f1_score']
display(scores)

{'f1_score': 0.03208152481600302,
 'precision_recall_curve': (array([7.24239548e-04, 1.63997685e-02, 1.00000000e+00]),
  array([1.        , 0.73275862, 0.        ]),
  array([0., 1.])),
 'recall_score': 0.7327586206896551,
 'classification_report': '              precision    recall  f1-score   support\n\n         0.0       1.00      0.97      0.98    160052\n         1.0       0.02      0.73      0.03       116\n\n    accuracy                           0.97    160168\n   macro avg       0.51      0.85      0.51    160168\nweighted avg       1.00      0.97      0.98    160168\n',
 'precision_score': 0.01639976847385684,
 'accuracy_score': 0.9679773737575545,
 'balanced_accuracy_score': 0.8504532363188859}

In [18]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [False],
    'max_depth': [50, 60, 70, 80],
    'max_features': [2, 3],
    'min_samples_leaf': [1, 2, 3],
    'min_samples_split': [4, 5, 16],
    'n_estimators': [500, 600, 700 ]
}
# Create a based model
clf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = clf, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)

In [ ]:
grid_search.fit(X_train,y_train)
grid_search.best_params_